In [1]:
import os

In [2]:
%pwd

'c:\\Users\\gokul\\OneDrive\\Desktop\\Projects\\Fashion_Recommender_with_MLOps_EndtoEnd\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\gokul\\OneDrive\\Desktop\\Projects\\Fashion_Recommender_with_MLOps_EndtoEnd'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ExtractionConfig:
    features_root_dir: Path
    files_root_dir: Path
    training_data: Path
    updated_base_model_path: Path

In [6]:
from myFashionRecommender.constants import *
from myFashionRecommender.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_extraction_config(self):
        config = self.config.feature_extraction
        model_config = self.config.prepare_base_model

        create_directories([config.features_root_dir])
        create_directories([config.files_root_dir])

        extraction_config = ExtractionConfig(
            features_root_dir = config.features_root_dir,
            files_root_dir = config.files_root_dir,
            training_data = config.local_data_file,
            updated_base_model_path = model_config.updated_base_model_path,
            
        )
        return extraction_config



In [8]:
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing import image
import numpy as np
from numpy.linalg import norm
from tqdm import tqdm
from tensorflow.keras.models import load_model
import pickle
from PIL import Image

In [9]:
class FeatureExtraction:
    def __init__(self, config: ExtractionConfig):
        self.config = config

    def extract_features(self,img_path,model):
        img = image.load_img(img_path,target_size=(224,224))
        img_array = image.img_to_array(img)
        expanded_img_array = np.expand_dims(img_array, axis=0)
        preprocessed_img = preprocess_input(expanded_img_array)
        result = model.predict(preprocessed_img).flatten()
        normalized_result = result / norm(result)

        return normalized_result
    
    def main_extraction(self):
        filenames = []

        for file in os.listdir(self.config.training_data):
            filenames.append(os.path.join(self.config.training_data,file))

        feature_list = []
        model = load_model(self.config.updated_base_model_path)

        for file in filenames:
            feature_list.append(self.extract_features(file,model))

        feature_list_path = os.path.join(self.config.features_root_dir, 'model.pkl')
        filenames_path = os.path.join(self.config.files_root_dir, 'filenames.pkl')

        pickle.dump(feature_list, open(feature_list_path, 'wb'))
        pickle.dump(filenames, open(filenames_path, 'wb'))
    

In [10]:
try:
    config_manager = ConfigurationManager()
    extraction_config = config_manager.get_extraction_config()
    feature_extraction = FeatureExtraction(extraction_config)
    feature_extraction.main_extraction()
except Exception as e:
    raise e

[2024-07-06 01:41:54,491: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-07-06 01:41:54,500: INFO: common: yaml file: params.yaml loaded successfully]
[2024-07-06 01:41:54,501: INFO: common: created directory at: artifacts]
[2024-07-06 01:41:54,502: INFO: common: created directory at: artifacts/features]
[2024-07-06 01:41:54,503: INFO: common: created directory at: artifacts/filenames]


[2024-07-06 01:41:56,903: WARNING: hdf5_format: No training configuration found in the save file, so the model was *not* compiled. Compile it manually.]
1/1 [==============================] - 0s 116ms/step
